# Cube and Data Warehouse mini Demo
This aims to implement a cube and do presentation
## Cube

In [1]:
import atoti as tt

Welcome to Atoti 0.8.10!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

Atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, you can request a trial license at https://www.atoti.io/evaluation-license-request.

You can hide this message by setting the `ATOTI_HIDE_EULA_MESSAGE` environment variable to True.


In [2]:
session = tt.Session(
    user_content_storage=".content",
    port=9092,
    java_options=["-Xms1G", "-Xmx10G"]
)

In [3]:
db_name = "AdventureworksDWDemo"
db_user = "postgres"
db_password = "postgres"  # Update with your password
db_host = "pgdb"  # Update if your DB is hosted elsewhere
db_port = "5432"

jdbc_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}?user={db_user}&password={db_password}"

In [4]:
product_sales_table = session.read_sql(
    "SELECT * FROM factproductsales",
    keys=["transactionid"],
    table_name="ProductSales",
    url=jdbc_url,
)

In [5]:
customer_table = session.read_sql(
    "SELECT * FROM dimcustomer",
    keys=["customerid"],
    table_name="Customer",
    url=jdbc_url)

In [6]:
product_table = session.read_sql(
    "SELECT * FROM dimproduct",
    keys=["productkey"],
    table_name="Product",
    url=jdbc_url
)

In [7]:
sales_person_table = session.read_sql(
    "SELECT * FROM dimsalesperson",
    keys=["salespersonid"],
    table_name="SalesPerson",
    url=jdbc_url
)

In [8]:
stores_table = session.read_sql(
    "SELECT * FROM dimstores",
    keys=["storeid"],
    table_name="Stores",
    url=jdbc_url
)

In [9]:
product_sales_table.head()

,salesinvoicenumber,storeid,customerid,productid,salespersonid,quantity,salestotalcost,productactualcost,deviation
transactionid,,,,,,,,,
7,3,1,3,3,2,1.0,43.5,42.0,1.5
15,6,1,3,5,2,1.0,139.0,135.0,4.0
23,10,1,3,4,2,3.0,60.0,54.0,6.0
8,3,1,3,4,2,3.0,60.0,54.0,6.0
16,7,2,1,4,3,3.0,60.0,54.0,6.0


In [10]:
product_sales_table.join(customer_table, product_sales_table["customerid"] == customer_table["customerid"])

In [11]:
product_sales_table.join(product_table, product_sales_table["productid"] == product_table["productkey"])

In [12]:
product_sales_table.join(sales_person_table,
                         product_sales_table["salespersonid"] == sales_person_table["salespersonid"])

In [13]:
product_sales_table.join(stores_table, product_sales_table["storeid"] == stores_table["storeid"])

In [14]:
session.tables.schema

```mermaid
erDiagram
  "Customer" {
    _ int PK "customerid"
    _ String "customeraltid"
    _ String "customername"
    _ String "gender"
  }
  "SalesPerson" {
    _ int PK "salespersonid"
    _ String "salespersonaltid"
    _ String "salespersonname"
    nullable int "storeid"
    _ String "city"
    _ String "state"
    _ String "country"
  }
  "Product" {
    _ int PK "productkey"
    _ String "productaltkey"
    _ String "productname"
    nullable double "productactualcost"
    nullable double "productsalescost"
  }
  "Stores" {
    _ int PK "storeid"
    _ String "storealtid"
    _ String "storename"
    _ String "storelocation"
    _ String "city"
    _ String "state"
    _ String "country"
  }
  "ProductSales" {
    _ long PK "transactionid"
    nullable int "salesinvoicenumber"
    nullable int "storeid"
    nullable int "customerid"
    nullable int "productid"
    nullable int "salespersonid"
    nullable double "quantity"
    nullable double "salestotalcost"
    nullable double "productactualcost"
    nullable double "deviation"
  }
  "ProductSales" }o--o| "Product" : "`productid` == `productkey`"
  "ProductSales" }o--o| "Customer" : "`customerid` == `customerid`"
  "ProductSales" }o--o| "SalesPerson" : "`salespersonid` == `salespersonid`"
  "ProductSales" }o--o| "Stores" : "`storeid` == `storeid`"
```


In [15]:
cube = session.create_cube(product_sales_table)

In [16]:
cube

In [17]:
hierarchies, levels, measures = cube.hierarchies, cube.levels, cube.measures

In [18]:
# original hierarchies
hierarchies

{('Stores', 'storename'): <atoti.hierarchy.Hierarchy object at 0x7fd665705550>, ('Stores', 'state'): <atoti.hierarchy.Hierarchy object at 0x7fd665718cd0>, ('Product', 'productname'): <atoti.hierarchy.Hierarchy object at 0x7fd665718e10>, ('SalesPerson', 'city'): <atoti.hierarchy.Hierarchy object at 0x7fd665718f10>, ('Product', 'productaltkey'): <atoti.hierarchy.Hierarchy object at 0x7fd665719010>, ('SalesPerson', 'salespersonaltid'): <atoti.hierarchy.Hierarchy object at 0x7fd665719110>, ('Stores', 'storealtid'): <atoti.hierarchy.Hierarchy object at 0x7fd665719210>, ('Stores', 'city'): <atoti.hierarchy.Hierarchy object at 0x7fd665719310>, ('ProductSales', 'transactionid'): <atoti.hierarchy.Hierarchy object at 0x7fd665719410>, ('Customer', 'customername'): <atoti.hierarchy.Hierarchy object at 0x7fd665719510>, ('SalesPerson', 'salespersonname'): <atoti.hierarchy.Hierarchy object at 0x7fd665719610>, ('SalesPerson', 'country'): <atoti.hierarchy.Hierarchy object at 0x7fd665719750>, ('Customer', 'customeraltid'): <atoti.hierarchy.Hierarchy object at 0x7fd665719890>, ('Stores', 'storelocation'): <atoti.hierarchy.Hierarchy object at 0x7fd6657199d0>, ('SalesPerson', 'state'): <atoti.hierarchy.Hierarchy object at 0x7fd665719b10>, ('Stores', 'country'): <atoti.hierarchy.Hierarchy object at 0x7fd665719c50>, ('Customer', 'gender'): <atoti.hierarchy.Hierarchy object at 0x7fd665719d90>}

In [19]:
# original levels
levels

In [20]:
levels

In [21]:
# dimensions
# Customer: CustomerName, Gender
# SalesPerson: SalesPersonName, City, State, Country
# Product: ProductName
# Store: StoreName, City, State

In [22]:
hierarchies["Customer"] = [levels["customername"], levels["gender"]]

In [23]:
hierarchies["Product"] = [levels["productname"]]

In [24]:
hierarchies["SalesPerson"] = [levels["salespersonname"], levels[('SalesPerson', 'city', 'city')],
                              levels[('SalesPerson', 'state', 'state')], levels[('SalesPerson', 'country', 'country')]]

In [25]:
hierarchies["Stores"] = [levels[('Stores', 'storename', 'storename')], levels[('Stores', 'city', 'city')],
                         levels[('Stores', 'state', 'state')], levels[('Stores', 'country', 'country')]]

In [26]:
hierarchies

{('Stores', 'storename'): <atoti.hierarchy.Hierarchy object at 0x7fd665743350>, ('Stores', 'state'): <atoti.hierarchy.Hierarchy object at 0x7fd66573ded0>, ('Stores', 'Stores'): <atoti.hierarchy.Hierarchy object at 0x7fd6657055d0>, ('Customer', 'Customer'): <atoti.hierarchy.Hierarchy object at 0x7fd665707050>, ('Product', 'productname'): <atoti.hierarchy.Hierarchy object at 0x7fd6657052d0>, ('SalesPerson', 'city'): <atoti.hierarchy.Hierarchy object at 0x7fd665706e90>, ('Product', 'productaltkey'): <atoti.hierarchy.Hierarchy object at 0x7fd665706550>, ('SalesPerson', 'salespersonaltid'): <atoti.hierarchy.Hierarchy object at 0x7fd665704610>, ('Stores', 'storealtid'): <atoti.hierarchy.Hierarchy object at 0x7fd665705490>, ('Stores', 'city'): <atoti.hierarchy.Hierarchy object at 0x7fd6657069d0>, ('ProductSales', 'transactionid'): <atoti.hierarchy.Hierarchy object at 0x7fd665705650>, ('Customer', 'customername'): <atoti.hierarchy.Hierarchy object at 0x7fd665705d50>, ('SalesPerson', 'SalesPerson'): <atoti.hierarchy.Hierarchy object at 0x7fd665707490>, ('SalesPerson', 'salespersonname'): <atoti.hierarchy.Hierarchy object at 0x7fd66c08d090>, ('SalesPerson', 'country'): <atoti.hierarchy.Hierarchy object at 0x7fd66c08d010>, ('Product', 'Product'): <atoti.hierarchy.Hierarchy object at 0x7fd66c08d650>, ('Customer', 'customeraltid'): <atoti.hierarchy.Hierarchy object at 0x7fd66c08d310>, ('Stores', 'storelocation'): <atoti.hierarchy.Hierarchy object at 0x7fd66c08ee50>, ('SalesPerson', 'state'): <atoti.hierarchy.Hierarchy object at 0x7fd66578b310>, ('Stores', 'country'): <atoti.hierarchy.Hierarchy object at 0x7fd66578a1d0>, ('Customer', 'gender'): <atoti.hierarchy.Hierarchy object at 0x7fd66578be90>}

In [27]:
# clean customer
del hierarchies[('Customer', 'customeraltid')]
del hierarchies[('Customer', 'customername')]
del hierarchies[('Customer', 'gender')]

In [28]:
# clean product
del hierarchies[('Product', 'productname')]
del hierarchies[('Product', 'productaltkey')]

In [29]:
# clean salesperson
del hierarchies[('SalesPerson', 'city')]
del hierarchies[('SalesPerson', 'salespersonname')]
del hierarchies[('SalesPerson', 'country')]
del hierarchies[('SalesPerson', 'state')]
del hierarchies[('SalesPerson', 'salespersonaltid')]

In [30]:
# clean stores
del hierarchies[('Stores', 'storename')]
del hierarchies[('Stores', 'state')]
del hierarchies[('Stores', 'country')]
del hierarchies[('Stores', 'storealtid')]
del hierarchies[('Stores', 'city')]

In [31]:
# clean product sales
del hierarchies[('ProductSales', 'transactionid')]

In [32]:
hierarchies

{('SalesPerson', 'SalesPerson'): <atoti.hierarchy.Hierarchy object at 0x7fd66c0bf690>, ('Stores', 'Stores'): <atoti.hierarchy.Hierarchy object at 0x7fd66c0d6d90>, ('Product', 'Product'): <atoti.hierarchy.Hierarchy object at 0x7fd66c0ceb50>, ('Customer', 'Customer'): <atoti.hierarchy.Hierarchy object at 0x7fd66c0ccd50>, ('Stores', 'storelocation'): <atoti.hierarchy.Hierarchy object at 0x7fd66c0ce210>}

In [33]:
# clean measures
del measures["contributors.COUNT"]
del measures["customerid.MEAN"]
del measures["customerid.SUM"]
del measures["deviation.MEAN"]
del measures["deviation.SUM"]
del measures["productactualcost.MEAN"]
del measures["productid.MEAN"]
del measures["productid.SUM"]
del measures["salesinvoicenumber.MEAN"]
del measures["salesinvoicenumber.SUM"]
del measures["salespersonid.MEAN"]
del measures["salespersonid.SUM"]
del measures["storeid.MEAN"]
del measures["storeid.SUM"]
del measures["salestotalcost.MEAN"]
del measures["quantity.MEAN"]

In [34]:
measures

{'update.TIMESTAMP': <atoti.measure.Measure object at 0x7fd665797b50>, 'quantity.SUM': <atoti.measure.Measure object at 0x7fd665795f90>, 'salestotalcost.SUM': <atoti.measure.Measure object at 0x7fd665797d10>, 'productactualcost.SUM': <atoti.measure.Measure object at 0x7fd665797f50>}

In [35]:
cube.query(measures["quantity.SUM"], levels=[levels[('Stores', 'Stores', 'city')]])

,,quantity.SUM
storename,city,
X-Mart,Ahmedabad,43.00


In [36]:
cost = tt.agg.sum(
    measures["quantity.SUM"] * tt.agg.single_value(product_table["productactualcost"]),
    scope=tt.OriginScope(levels["productname"]),
)

income = tt.agg.sum(
    measures["quantity.SUM"] * tt.agg.single_value(product_table["productsalescost"]),
    scope=tt.OriginScope(levels["productname"]),
)


/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_dataclasses.py:135: FutureWarning: The variadic constructor is deprecated, pass a named `levels` parameter instead.
  s.__pydantic_validator__.validate_python(ArgsKwargs(args, kwargs), self_instance=s)


In [37]:
measures["Margin"] = income - cost

In [38]:
cube.query(measures["Margin"], levels=[levels[('Product', 'Product', 'productname')]])

,Margin
productname,
Arial Washing Powder 1kg,12.00
Nirma Soap,36.00
Rice Grains 1kg,12.00
SunFlower Oil 1 ltr,10.50
Wheat Floor 1kg,7.00


## Presentation

In [39]:
session.widget

Open the notebook in JupyterLab with the Atoti JupyterLab extension enabled to build this widget.

In [40]:
session.link

http://localhost:9092

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.

## What-if Analysis

In [41]:
import pandas as pd
from sqlalchemy import create_engine

In [42]:
connection_url = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create the engine
engine = create_engine(connection_url)

In [43]:
product_df = pd.read_sql_query("SELECT * FROM dimproduct", engine)

In [44]:
product_df

,productkey,productaltkey,productname,productactualcost,productsalescost
0,1,ITM-001,Wheat Floor 1kg,5.5,6.5
1,2,ITM-002,Rice Grains 1kg,22.5,24.0
2,3,ITM-003,SunFlower Oil 1 ltr,42.0,43.5
3,4,ITM-004,Nirma Soap,18.0,20.0
4,5,ITM-005,Arial Washing Powder 1kg,135.0,139.0


In [45]:
better_actual_cost = {
    "ITM-001": 4.5,
    "ITM-002": 10,
    "ITM-003": 30,
    "ITM-004": 15,
    "ITM-005": 135,
}

In [46]:
for product, actual_cost in better_actual_cost.items():
    product_df.loc[
        product_df["productaltkey"] == product, "productactualcost"
    ] = actual_cost
product_df

,productkey,productaltkey,productname,productactualcost,productsalescost
0,1,ITM-001,Wheat Floor 1kg,4.5,6.5
1,2,ITM-002,Rice Grains 1kg,10.0,24.0
2,3,ITM-003,SunFlower Oil 1 ltr,30.0,43.5
3,4,ITM-004,Nirma Soap,15.0,20.0
4,5,ITM-005,Arial Washing Powder 1kg,135.0,139.0


In [47]:
product_table.scenarios["Lower actual cost"].load_pandas(product_df)